In [ ]:
# Sources
# https://www.statsmodels.org/dev/generated/statsmodels.stats.mediation.Mediation.html

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statistics
from scipy import stats
from sklearn.impute import SimpleImputer
from statsmodels.stats.mediation import Mediation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Junior/Dyanne JP/ABCD_Release4.0_Tabular_Dataset.csv')
df_baseline = df[df['eventname'] == 'baseline_year_1_arm_1']
df_2year = df[df['eventname'] == '2_year_follow_up_y_arm_1']

In [ ]:
baseline_2year = pd.merge(df_baseline, df_2year, on='subjectkey', how='left', suffixes=('_baseline', '_2year'))

In [ ]:
baseline_2year.dropna(subset=['eventname_2year'], inplace=True)

In [ ]:
check = ['nihtbx_picvocab_uncorrected_baseline','nihtbx_picvocab_uncorrected_2year','nihtbx_flanker_uncorrected_baseline',
         'nihtbx_flanker_uncorrected_2year','nihtbx_pattern_uncorrected_baseline','nihtbx_pattern_uncorrected_2year',
         'nihtbx_picture_uncorrected_baseline','nihtbx_picture_uncorrected_2year','nihtbx_reading_uncorrected_baseline',
         'nihtbx_reading_uncorrected_2year','nihtbx_cryst_uncorrected_baseline','nihtbx_cryst_uncorrected_2year']

cleaned = baseline_2year.dropna(subset=check)
#(7172,1055)

# drop columns that only have NaN values
cleaned.dropna(axis=1, how='all', inplace=True)
#(7172,892)

print('Number of participants with all 5 test scores')
print(cleaned.shape[0])

In [ ]:
non_numeric_columns = cleaned.select_dtypes(exclude=['number']).columns

print("Non-numeric columns:", non_numeric_columns.tolist())

In [ ]:
(cleaned['sex_baseline']==cleaned['sex_2year']).sum()

In [ ]:
cleaned = cleaned.drop('sex_2year',axis=1)

In [ ]:
cleaned['sex_baseline'] = cleaned['sex_baseline'].replace({'M': 0, 'F': 1})

In [ ]:
def medianimpute(df):
    # Create a copy of the DataFrame to avoid changing the original data
    df_imputed = df.copy()

    # Identify numeric columns by data type
    numeric_cols = df_imputed.select_dtypes(include=[np.number]).columns

    # Define the imputer with a median strategy
    imputer = SimpleImputer(strategy='median')

    # Apply the imputer only to the numeric columns
    df_imputed[numeric_cols] = imputer.fit_transform(df_imputed[numeric_cols])

    return df_imputed

In [ ]:
imputed = medianimpute(cleaned)

In [ ]:
# # nest family in ABCD study site for LME random effect
# imputed['site_family'] = imputed['abcd_site_baseline'].astype(str) + "_" + imputed['family_id_baseline'].astype(str)
# imputed['site_family'] = imputed['abcd_site_2year'].astype(str) + "_" + imputed['family_id_baseline'].astype(str)

In [ ]:
df_low = imputed[(imputed['income_baseline'] >= 1) & (imputed['income_baseline'] < 7)]
df_med = imputed[imputed['income_baseline'].isin([7])]
df_high = imputed[(imputed['income_baseline'] > 7) & (imputed['income_baseline'] <= 10)]

In [ ]:
low_diff = pd.DataFrame({
    'low_diff_picvocab': df_low['nihtbx_picvocab_uncorrected_2year'] - df_low['nihtbx_picvocab_uncorrected_baseline'],
    'low_diff_flanker': df_low['nihtbx_flanker_uncorrected_2year'] - df_low['nihtbx_flanker_uncorrected_baseline'],
    'low_diff_pattern': df_low['nihtbx_pattern_uncorrected_2year'] - df_low['nihtbx_pattern_uncorrected_baseline'],
    'low_diff_picture': df_low['nihtbx_picture_uncorrected_2year'] - df_low['nihtbx_picture_uncorrected_baseline'],
    'low_diff_reading': df_low['nihtbx_reading_uncorrected_2year'] - df_low['nihtbx_reading_uncorrected_baseline']
})

high_diff = pd.DataFrame({
    'high_diff_picvocab': df_high['nihtbx_picvocab_uncorrected_2year'] - df_high['nihtbx_picvocab_uncorrected_baseline'],
    'high_diff_flanker': df_high['nihtbx_flanker_uncorrected_2year'] - df_high['nihtbx_flanker_uncorrected_baseline'],
    'high_diff_pattern': df_high['nihtbx_pattern_uncorrected_2year'] - df_high['nihtbx_pattern_uncorrected_baseline'],
    'high_diff_picture': df_high['nihtbx_picture_uncorrected_2year'] - df_high['nihtbx_picture_uncorrected_baseline'],
    'high_diff_reading': df_high['nihtbx_reading_uncorrected_2year'] - df_high['nihtbx_reading_uncorrected_baseline']
})

df_low = pd.concat([df_low, low_diff], axis=1)
df_high = pd.concat([df_high, high_diff], axis=1)

In [ ]:
low_picvocab_neg = df_low[df_low['low_diff_picvocab']<0]
high_picvocab_neg = df_high[df_high['high_diff_picvocab']<0]
low_picvocab_pos = df_low[df_low['low_diff_picvocab']>0]
high_picvocab_pos = df_high[df_high['high_diff_picvocab']>0]

low_flanker_neg = df_low[df_low['low_diff_flanker']<0]
high_flanker_neg = df_high[df_high['high_diff_flanker']<0]
low_flanker_pos = df_low[df_low['low_diff_flanker']>0]
high_flanker_pos = df_high[df_high['high_diff_flanker']>0]

low_pattern_neg = df_low[df_low['low_diff_pattern']<0]
high_pattern_neg = df_high[df_high['high_diff_pattern']<0]
low_pattern_pos = df_low[df_low['low_diff_pattern']>0]
high_pattern_pos = df_high[df_high['high_diff_pattern']>0]

low_picture_neg = df_low[df_low['low_diff_picture']<0]
high_picture_neg = df_high[df_high['high_diff_picture']<0]
low_picture_pos = df_low[df_low['low_diff_picture']>0]
high_picture_pos = df_high[df_high['high_diff_picture']>0]

low_reading_neg = df_low[df_low['low_diff_reading']<0]
high_reading_neg = df_high[df_high['high_diff_reading']<0]
low_reading_pos = df_low[df_low['low_diff_reading']>0]
high_reading_pos = df_high[df_high['high_diff_reading']>0]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picvocab_uncorrected_2year ~ ELS_total_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('ELS_total_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'ELS_total_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('ELS_total_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picvocab_uncorrected_2year ~ ELS_total_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('ELS_total_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'ELS_total_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('ELS_total_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picvocab_uncorrected_2year ~ ELS_total_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('ELS_total_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'ELS_total_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('ELS_total_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picvocab_uncorrected_2year ~ ELS_total_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('ELS_total_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'ELS_total_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('ELS_total_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picvocab_uncorrected_2year ~ neuroticismauto_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('neuroticismauto_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'neuroticismauto_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('neuroticismauto_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picvocab_uncorrected_2year ~ neuroticismauto_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('neuroticismauto_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'neuroticismauto_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('neuroticismauto_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picvocab_uncorrected_2year ~ neuroticismauto_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('neuroticismauto_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'neuroticismauto_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('neuroticismauto_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picvocab_uncorrected_2year ~ neuroticismauto_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('neuroticismauto_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'neuroticismauto_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('neuroticismauto_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picvocab_uncorrected_2year ~ screentime_maturegames_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('screentime_maturegames_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'screentime_maturegames_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('screentime_maturegames_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picvocab_uncorrected_2year ~ screentime_maturegames_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('screentime_maturegames_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'screentime_maturegames_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('screentime_maturegames_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picvocab_uncorrected_2year ~ screentime_maturegames_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('screentime_maturegames_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'screentime_maturegames_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('screentime_maturegames_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picvocab_uncorrected_2year ~ screentime_maturegames_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('screentime_maturegames_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'screentime_maturegames_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('screentime_maturegames_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picvocab_uncorrected_2year ~ screentime_wkday_p_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('screentime_wkday_p_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'screentime_wkday_p_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('screentime_wkday_p_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picvocab_uncorrected_2year ~ screentime_wkday_p_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('screentime_wkday_p_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'screentime_wkday_p_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('screentime_wkday_p_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picvocab_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picvocab_uncorrected_2year ~ screentime_wkday_p_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('screentime_wkday_p_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'screentime_wkday_p_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('screentime_wkday_p_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picvocab_uncorrected_2year ~ screentime_wkday_p_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('screentime_wkday_p_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'screentime_wkday_p_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('screentime_wkday_p_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picvocab_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_flanker_uncorrected_2year ~ good_school_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('good_school_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'good_school_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('good_school_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_flanker_uncorrected_2year ~ good_school_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('good_school_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'good_school_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('good_school_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_flanker_uncorrected_2year ~ good_school_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('good_school_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'good_school_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('good_school_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_flanker_uncorrected_2year ~ good_school_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('good_school_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'good_school_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('good_school_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_flanker_uncorrected_2year ~ bpm_total_t_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('bpm_total_t_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'bpm_total_t_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('bpm_total_t_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_flanker_uncorrected_2year ~ bpm_total_t_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('bpm_total_t_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'bpm_total_t_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('bpm_total_t_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_flanker_uncorrected_2year ~ bpm_total_t_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('bpm_total_t_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'bpm_total_t_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('bpm_total_t_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_flanker_uncorrected_2year ~ bpm_total_t_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('bpm_total_t_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'bpm_total_t_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('bpm_total_t_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_flanker_uncorrected_2year ~ Cfriends_same_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('Cfriends_same_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'Cfriends_same_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('Cfriends_same_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_flanker_uncorrected_2year ~ Cfriends_same_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('Cfriends_same_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'Cfriends_same_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('Cfriends_same_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_flanker_uncorrected_2year ~ Cfriends_same_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('Cfriends_same_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'Cfriends_same_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('Cfriends_same_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_flanker_uncorrected_2year ~ Cfriends_same_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('Cfriends_same_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'Cfriends_same_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('Cfriends_same_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_flanker_uncorrected_2year ~ cbcl_totprob_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('cbcl_totprob_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'cbcl_totprob_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('cbcl_totprob_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_flanker_uncorrected_2year ~ cbcl_totprob_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('cbcl_totprob_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'cbcl_totprob_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('cbcl_totprob_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_flanker_uncorrected_2year ~ cbcl_totprob_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('cbcl_totprob_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'cbcl_totprob_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('cbcl_totprob_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_flanker_uncorrected_2year ~ cbcl_totprob_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('cbcl_totprob_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'cbcl_totprob_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('cbcl_totprob_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_flanker_uncorrected_2year ~ good_school_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('good_school_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'good_school_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('good_school_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_flanker_uncorrected_2year ~ good_school_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('good_school_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'good_school_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('good_school_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_flanker_uncorrected_2year ~ good_school_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('good_school_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'good_school_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('good_school_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_flanker_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_flanker_uncorrected_2year ~ good_school_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('good_school_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'good_school_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('good_school_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_flanker_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_pattern_uncorrected_2year ~ screentime_wkend_y_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('screentime_wkend_y_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'screentime_wkend_y_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('screentime_wkend_y_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_pattern_uncorrected_2year ~ screentime_wkend_y_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('screentime_wkend_y_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'screentime_wkend_y_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('screentime_wkend_y_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_pattern_uncorrected_2year ~ screentime_wkend_y_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('screentime_wkend_y_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'screentime_wkend_y_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('screentime_wkend_y_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_pattern_uncorrected_2year ~ screentime_wkend_y_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('screentime_wkend_y_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'screentime_wkend_y_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('screentime_wkend_y_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_pattern_uncorrected_2year ~ cbcl_totprob_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('cbcl_totprob_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'cbcl_totprob_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('cbcl_totprob_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_pattern_uncorrected_2year ~ cbcl_totprob_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('cbcl_totprob_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'cbcl_totprob_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('cbcl_totprob_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_pattern_uncorrected_2year ~ cbcl_totprob_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('cbcl_totprob_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'cbcl_totprob_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('cbcl_totprob_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_pattern_uncorrected_2year ~ cbcl_totprob_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('cbcl_totprob_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'cbcl_totprob_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('cbcl_totprob_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_pattern_uncorrected_2year ~ cpeur2_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('cpeur2_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'cpeur2_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('cpeur2_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_pattern_uncorrected_2year ~ cpeur2_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('cpeur2_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'cpeur2_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('cpeur2_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_pattern_uncorrected_2year ~ cpeur2_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('cpeur2_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'cpeur2_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('cpeur2_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_pattern_uncorrected_2year ~ cpeur2_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('cpeur2_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'cpeur2_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('cpeur2_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_pattern_uncorrected_2year ~ screentime_wkend_tv_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('screentime_wkend_tv_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'screentime_wkend_tv_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('screentime_wkend_tv_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_pattern_uncorrected_2year ~ screentime_wkend_tv_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('screentime_wkend_tv_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'screentime_wkend_tv_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('screentime_wkend_tv_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_pattern_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_pattern_uncorrected_2year ~ screentime_wkend_tv_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('screentime_wkend_tv_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'screentime_wkend_tv_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('screentime_wkend_tv_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_pattern_uncorrected_2year ~ screentime_wkend_tv_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('screentime_wkend_tv_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'screentime_wkend_tv_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('screentime_wkend_tv_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_pattern_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picture_uncorrected_2year ~ cbcl_external_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picture_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('cbcl_external_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picture_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'cbcl_external_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('cbcl_external_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picture_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picture_uncorrected_2year ~ cbcl_external_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picture_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('cbcl_external_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picture_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'cbcl_external_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('cbcl_external_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picture_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picture_uncorrected_2year ~ cbcl_external_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picture_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('cbcl_external_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picture_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'cbcl_external_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('cbcl_external_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picture_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picture_uncorrected_2year ~ cbcl_external_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picture_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('cbcl_external_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picture_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'cbcl_external_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('cbcl_external_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picture_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picture_uncorrected_2year ~ parent_age_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picture_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('parent_age_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picture_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'parent_age_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('parent_age_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picture_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picture_uncorrected_2year ~ parent_age_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picture_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('parent_age_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picture_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'parent_age_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('parent_age_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picture_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picture_uncorrected_2year ~ parent_age_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picture_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('parent_age_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picture_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'parent_age_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('parent_age_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picture_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picture_uncorrected_2year ~ parent_age_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picture_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('parent_age_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picture_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'parent_age_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('parent_age_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picture_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picture_uncorrected_2year ~ BAS_Drive_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picture_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('BAS_Drive_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picture_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'BAS_Drive_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('BAS_Drive_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picture_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picture_uncorrected_2year ~ BAS_Drive_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picture_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('BAS_Drive_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picture_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'BAS_Drive_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('BAS_Drive_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_picture_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picture_uncorrected_2year ~ BAS_Drive_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picture_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('BAS_Drive_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picture_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'BAS_Drive_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('BAS_Drive_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picture_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picture_uncorrected_2year ~ BAS_Drive_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picture_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('BAS_Drive_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picture_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'BAS_Drive_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('BAS_Drive_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_picture_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_reading_uncorrected_2year ~ parent_age_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('parent_age_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'parent_age_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('parent_age_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_reading_uncorrected_2year ~ parent_age_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('parent_age_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'parent_age_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('parent_age_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_reading_uncorrected_2year ~ parent_age_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('parent_age_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'parent_age_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('parent_age_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_reading_uncorrected_2year ~ parent_age_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('parent_age_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'parent_age_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('parent_age_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_reading_uncorrected_2year ~ bpm_total_t_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('bpm_total_t_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'bpm_total_t_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('bpm_total_t_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_reading_uncorrected_2year ~ bpm_total_t_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('bpm_total_t_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'bpm_total_t_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('bpm_total_t_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_reading_uncorrected_2year ~ bpm_total_t_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('bpm_total_t_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'bpm_total_t_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('bpm_total_t_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_reading_uncorrected_2year ~ bpm_total_t_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('bpm_total_t_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'bpm_total_t_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('bpm_total_t_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_reading_uncorrected_2year ~ ELS_total_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('ELS_total_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'ELS_total_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('ELS_total_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_reading_uncorrected_2year ~ ELS_total_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('ELS_total_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'ELS_total_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('ELS_total_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_reading_uncorrected_2year ~ ELS_total_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('ELS_total_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'ELS_total_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('ELS_total_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_reading_uncorrected_2year ~ ELS_total_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('ELS_total_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'ELS_total_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('ELS_total_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_reading_uncorrected_2year ~ height_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('height_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'height_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('height_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_reading_uncorrected_2year ~ height_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('height_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'height_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('height_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=low_reading_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_reading_uncorrected_2year ~ height_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_neg)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('height_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_neg)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'height_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('height_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_neg).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_reading_uncorrected_2year ~ height_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_pos)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('height_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_pos)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'height_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('height_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=high_reading_pos).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picvocab_uncorrected_2year ~ ELS_total_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_low)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('ELS_total_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_low)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'ELS_total_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picvocab_uncorrected_2year ~ ELS_total_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_high)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('ELS_total_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_high)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'ELS_total_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_reading_uncorrected_2year ~ ELS_total_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_low)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('ELS_total_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_low)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'ELS_total_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_reading_uncorrected_2year ~ ELS_total_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_high)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('ELS_total_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_high)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'ELS_total_2year')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('ELS_total_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_low).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
model = smf.ols('ELS_total_2year ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_high).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picvocab_uncorrected_2year ~ screentime_wkday_p_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_low)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('screentime_wkday_p_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_low)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'screentime_wkday_p_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_picvocab_uncorrected_2year ~ screentime_wkday_p_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_high)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('screentime_wkday_p_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_high)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'screentime_wkday_p_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('screentime_wkday_p_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_low).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
model = smf.ols('screentime_wkday_p_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_high).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_flanker_uncorrected_2year ~ cbcl_totprob_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_low)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('cbcl_totprob_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_low)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'cbcl_totprob_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_flanker_uncorrected_2year ~ cbcl_totprob_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_high)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('cbcl_totprob_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_high)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'cbcl_totprob_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('cbcl_totprob_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_low).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
model = smf.ols('cbcl_totprob_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_high).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_reading_uncorrected_2year ~ parent_age_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_low)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('parent_age_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_low)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'parent_age_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
# Specify the model for the outcome
outcome_model = sm.OLS.from_formula('nihtbx_reading_uncorrected_2year ~ parent_age_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_high)

# Specify the model for the mediator
mediator_model = sm.OLS.from_formula('parent_age_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_high)

# Set up the mediation analysis model
med = Mediation(outcome_model, mediator_model, 'income_baseline', 'parent_age_baseline')

# Fit the mediation model
med_result = med.fit()

# Print the results
print(med_result.summary())

In [ ]:
model = smf.ols('parent_age_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_low).fit()

# Print the summary of the regression model
print(model.summary())

In [ ]:
model = smf.ols('parent_age_baseline ~ income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year', data=df_high).fit()

# Print the summary of the regression model
print(model.summary())